In [1]:
# =========================
# 1. IMPORT LIBRARIES
# =========================
import os
import random
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Subset
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights

import matplotlib.pyplot as plt

In [2]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score


In [10]:
# Set random seed untuk reproducibility
torch.manual_seed(42)
np.random.seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)

print(f"PyTorch Version: {torch.__version__}")
print(f"CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA Device: {torch.cuda.get_device_name(0)}")

PyTorch Version: 2.2.2+cpu
CUDA Available: False


In [4]:
# =========================
# 3. TRANSFORM / AUGMENTASI
# =========================

# Transform untuk TRAIN (dengan augmentasi)
train_transform = transforms.Compose([
    # 1) Sedikit geser & zoom tapi tetap 224x224
    transforms.RandomResizedCrop(
        size=224,
        scale=(0.9, 1.0),     # jangan terlalu kecil
        ratio=(0.9, 1.1),     # hampir square, aman buat wajah
    ),
    # 2) Flip kiri-kanan (pose terbalik dikit masih masuk akal)
    transforms.RandomHorizontalFlip(p=0.5),
    # 3) Rotasi kecil
    transforms.RandomRotation(degrees=10),
    # 4) Variasi cahaya & warna (ringan)
    transforms.ColorJitter(
        brightness=0.2,
        contrast=0.2,
        saturation=0.15,
        hue=0.02,
    ),
    # (Optional, boleh dibuang kalau takut terlalu agresif)
    # transforms.GaussianBlur(kernel_size=3, sigma=(0.1, 1.0)),

    # 5) Convert ke tensor + normalisasi ImageNet
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

# Transform untuk VALIDATION / TEST (TANPA augmentasi)
val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

In [5]:
# =========================
# 4. DATASET & DATALOADER
# =========================

data_root = "./train_face224"  # ganti jika foldermu beda

# base_dataset tanpa transform (untuk ambil indeks & nama kelas)
base_dataset = ImageFolder(data_root)
class_names = base_dataset.classes
num_classes = len(class_names)
print("Jumlah kelas:", num_classes)
print("Contoh kelas:", class_names[:5])

# Split train/val (80:20)
val_ratio = 0.2
num_samples = len(base_dataset)
indices = torch.randperm(num_samples).tolist()
split = int(num_samples * (1 - val_ratio))
train_indices, val_indices = indices[:split], indices[split:]

# Dataset dengan transform berbeda tapi memakai indeks yang sama
train_dataset_full = ImageFolder(data_root, transform=train_transform)
val_dataset_full   = ImageFolder(data_root, transform=val_transform)

train_dataset = Subset(train_dataset_full, train_indices)
val_dataset   = Subset(val_dataset_full,   val_indices)

print(f"Train samples: {len(train_dataset)}")
print(f"Val   samples: {len(val_dataset)}")

batch_size = 16

train_loader = DataLoader(train_dataset, batch_size=batch_size,
                          shuffle=True, num_workers=2, pin_memory=True)
val_loader   = DataLoader(val_dataset,   batch_size=batch_size,
                          shuffle=False, num_workers=2, pin_memory=True)

Jumlah kelas: 70
Contoh kelas: ['Abraham Ganda Napitu', 'Abu Bakar Siddiq Siregar', 'Ahmad Faqih Hasani', 'Aldi Sanjaya', 'Alfajar']
Train samples: 227
Val   samples: 57


In [6]:
# =========================
# 5. MODEL: EFFICIENTNET-B0
# =========================

# Pakai pretrained ImageNet
weights = EfficientNet_B0_Weights.IMAGENET1K_V1
model = efficientnet_b0(weights=weights)

# Ganti classifier jadi num_classes
in_features = model.classifier[1].in_features
model.classifier[1] = nn.Linear(in_features, num_classes)

model = model.to(device)

In [7]:
# =========================
# 6. LOSS, OPTIMIZER, SCHEDULER
# =========================

criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=1e-4,
    weight_decay=1e-4
)

# Turunkan learning rate kalau val_loss tidak membaik
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=0.5, patience=3, verbose=True
)

c:\Users\Asus\.conda\envs\DeepLearning\Lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


In [8]:
# =========================
# 7. FUNGSI TRAIN & VALIDATE
# =========================

def train_one_epoch(model, loader, optimizer, criterion, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for imgs, labels in loader:
        imgs = imgs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        preds = outputs.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    epoch_loss = running_loss / len(loader)
    epoch_acc = correct / total
    return epoch_loss, epoch_acc


def validate(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for imgs, labels in loader:
            imgs = imgs.to(device)
            labels = labels.to(device)

            outputs = model(imgs)
            loss = criterion(outputs, labels)

            running_loss += loss.item()
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    epoch_loss = running_loss / len(loader)
    epoch_acc = correct / total
    return epoch_loss, epoch_acc

In [9]:
# =========================
# 8. TRAINING LOOP + EARLY STOPPING
# =========================

num_epochs = 100
patience = 10          # berhenti kalau 10 epoch berturut-turut tidak membaik
best_val_loss = float("inf")
best_model_wts = None
epochs_no_improve = 0

for epoch in range(1, num_epochs + 1):
    train_loss, train_acc = train_one_epoch(model, train_loader, optimizer, criterion, device)
    val_loss, val_acc     = validate(model, val_loader, criterion, device)

    scheduler.step(val_loss)

    print(f"Epoch [{epoch}/{num_epochs}]")
    print(f"  Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}")
    print(f"  Val   Loss: {val_loss:.4f} | Val   Acc: {val_acc:.4f}")

    # Cek perbaikan untuk early stopping (berdasarkan val_loss)
    if val_loss < best_val_loss - 1e-4:   # min_delta kecil
        best_val_loss = val_loss
        best_model_wts = model.state_dict()
        epochs_no_improve = 0
        print("  -> Val loss membaik, simpan bobot terbaik sementara.")
    else:
        epochs_no_improve += 1
        print(f"  -> Tidak membaik selama {epochs_no_improve} epoch.")

        if epochs_no_improve >= patience:
            print("Early stopping aktif! Berhenti training.")
            break

# Setelah training, load bobot terbaik
if best_model_wts is not None:
    model.load_state_dict(best_model_wts)
else:
    print("Peringatan: best_model_wts None, gunakan bobot terakhir apa adanya.")

Epoch [1/100]
  Train Loss: 4.2448 | Train Acc: 0.0396
  Val   Loss: 4.2182 | Val   Acc: 0.0175
  -> Val loss membaik, simpan bobot terbaik sementara.
Epoch [2/100]
  Train Loss: 4.0463 | Train Acc: 0.1322
  Val   Loss: 4.1456 | Val   Acc: 0.1404
  -> Val loss membaik, simpan bobot terbaik sementara.


KeyboardInterrupt: 

In [ ]:
# =========================
# 9. EVALUASI LENGKAP (CONFUSION MATRIX, PRECISION, RECALL, F1)
# =========================

def evaluate_with_metrics(model, loader, class_names, device):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for imgs, labels in loader:
            imgs = imgs.to(device)
            labels = labels.to(device)

            outputs = model(imgs)
            preds = outputs.argmax(dim=1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)

    # Confusion Matrix
    cm = confusion_matrix(all_labels, all_preds)
    print("Confusion Matrix (raw counts):")
    print(cm)

    # Lebih enak kalau divisualisasikan
    plt.figure(figsize=(10, 8))
    im = plt.imshow(cm, interpolation='nearest')
    plt.title("Confusion Matrix")
    plt.colorbar(im, fraction=0.046, pad=0.04)
    tick_marks = np.arange(len(class_names))
    plt.xticks(tick_marks, class_names, rotation=90, fontsize=6)
    plt.yticks(tick_marks, class_names, fontsize=6)
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.tight_layout()
    plt.show()

    # Precision, Recall, F1 per kelas + macro/micro avg
    print("\nClassification Report:")
    print(classification_report(all_labels, all_preds,
                                target_names=class_names,
                                digits=4))

    # Kalau mau ambil nilai rata-rata:
    precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(
        all_labels, all_preds, average='macro', zero_division=0
    )
    precision_weighted, recall_weighted, f1_weighted, _ = precision_recall_fscore_support(
        all_labels, all_preds, average='weighted', zero_division=0
    )

    print(f"\nMacro Avg   - Precision: {precision_macro:.4f}, Recall: {recall_macro:.4f}, F1: {f1_macro:.4f}")
    print(f"Weighted Avg- Precision: {precision_weighted:.4f}, Recall: {recall_weighted:.4f}, F1: {f1_weighted:.4f}")

    return cm, (precision_macro, recall_macro, f1_macro)

# Panggil evaluasi di VAL set (bisa juga di test_loader kalau punya)
cm, macro_scores = evaluate_with_metrics(model, val_loader, class_names, device)

In [ ]:

# =========================
# 10. SAVE MODEL TERBAIK
# =========================

save_path = "efficientnet_b0_best.pth"
torch.save({
    "model_state_dict": model.state_dict(),
    "num_classes": num_classes,
    "class_names": class_names,
}, save_path)

print(f"Model terbaik disimpan ke: {save_path}")
